<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-нужных-библиотек-и-изучение-данных" data-toc-modified-id="Импорт-нужных-библиотек-и-изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт нужных библиотек и изучение данных</a></span></li><li><span><a href="#Разделение-датасата-на-выборки" data-toc-modified-id="Разделение-датасата-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение датасата на выборки</a></span></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение модели</a></span><ul class="toc-item"><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li></ul></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span><ul class="toc-item"><li><span><a href="#DecisionTree" data-toc-modified-id="DecisionTree-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>DecisionTree</a></span></li><li><span><a href="#RandomForest" data-toc-modified-id="RandomForest-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>RandomForest</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li></ul></div>

# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Импорт нужных библиотек и изучение данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод:** Датасет состоит из 3214 строк и 5 столбцов. Предобработка не требуется, пропусков нет, все данные в нужном формате.

## Разделение датасата на выборки

Для обучения, тестирования и проверки модели разделим данные на 3 выборки: тренировочную, тестовую и валидационную.

Для тренировочной выборки выделим 60% данных, для тестовой и валидационной по 20%

In [4]:
df_train, df_rest = train_test_split(df, test_size=0.4, random_state=1000)

In [5]:
df_test, df_valid = train_test_split(df_rest, test_size=0.5, random_state=1000)

In [6]:
df_train.shape

(1928, 5)

In [7]:
df_test.shape

(643, 5)

In [8]:
df_valid.shape

(643, 5)

**Вывод:** Разделили датасет на 3 выборки в соотношении 60/20/20:
* df_train - тренировочная выборка
* df_test - тестовая выборка
* df_valid - валидационная выборка

## Обучение модели

In [9]:
features_train = df_train.drop('is_ultra', axis = 1)
target_train = df_train['is_ultra']

In [10]:
features_valid = df_valid.drop('is_ultra', axis = 1)
target_valid = df_valid['is_ultra']

### DecisionTreeClassifier

In [11]:
for depth in range(3, 7):
    decision_model = DecisionTreeClassifier(random_state=1000, max_depth=depth)
    decision_model.fit(features_train, target_train)
    predictions_train = decision_model.predict(features_train)
    predictions_valid = decision_model.predict(features_valid)
    print('Accuracy при max_depth =', depth)
    print('Обучающая выборка:', accuracy_score(target_train, predictions_train))
    print('Валидационная выборка:', accuracy_score(target_valid, predictions_valid))
    print()

Accuracy при max_depth = 3
Обучающая выборка: 0.7904564315352697
Валидационная выборка: 0.8164852255054432

Accuracy при max_depth = 4
Обучающая выборка: 0.8049792531120332
Валидационная выборка: 0.8164852255054432

Accuracy при max_depth = 5
Обучающая выборка: 0.8091286307053942
Валидационная выборка: 0.8164852255054432

Accuracy при max_depth = 6
Обучающая выборка: 0.8200207468879668
Валидационная выборка: 0.8009331259720062



### LogisticRegression

In [12]:
logistic_model = LogisticRegression(random_state=1000)
logistic_model.fit(features_train, target_train)
predictions_train = logistic_model.predict(features_train)
predictions_valid = logistic_model.predict(features_valid)
print('Обучабщая выборка:', accuracy_score(target_train, predictions_train))
print('Валидационная выборка:', accuracy_score(target_valid, predictions_valid))


Обучабщая выборка: 0.7059128630705395
Валидационная выборка: 0.7045101088646968


### RandomForestClassifier

In [18]:
for estim in range(100, 110):
  forest_model = RandomForestClassifier(random_state=1000, n_estimators=estim)
  forest_model.fit(features_train, target_train)
  predictions_train = forest_model.predict(features_train)
  predictions_valid = forest_model.predict(features_valid)
  print('Accuracy при n_estimators = ', estim)
  print('Обучающая выборка:', accuracy_score(target_train, predictions_train))
  print('Валидационная выборка:', accuracy_score(target_valid, predictions_valid))
  print()

Accuracy при n_estimators =  100
Обучающая выборка: 1.0
Валидационная выборка: 0.8009331259720062

Accuracy при n_estimators =  101
Обучающая выборка: 1.0
Валидационная выборка: 0.80248833592535

Accuracy при n_estimators =  102
Обучающая выборка: 1.0
Валидационная выборка: 0.80248833592535

Accuracy при n_estimators =  103
Обучающая выборка: 1.0
Валидационная выборка: 0.8009331259720062

Accuracy при n_estimators =  104
Обучающая выборка: 1.0
Валидационная выборка: 0.8009331259720062

Accuracy при n_estimators =  105
Обучающая выборка: 1.0
Валидационная выборка: 0.8009331259720062

Accuracy при n_estimators =  106
Обучающая выборка: 1.0
Валидационная выборка: 0.8009331259720062

Accuracy при n_estimators =  107
Обучающая выборка: 1.0
Валидационная выборка: 0.80248833592535

Accuracy при n_estimators =  108
Обучающая выборка: 1.0
Валидационная выборка: 0.7993779160186625

Accuracy при n_estimators =  109
Обучающая выборка: 1.0
Валидационная выборка: 0.80248833592535



**Выводы:**
* Лучше всего отработали деревья решений
* При увеличении значения max_depth у модели дерева решений падает точность(будем использовать значение max_depth = 4)
* Модель RandomForestClassifier так же удволетворяет требованиям, будем испольвать ее со значением e_estimators = 100. 

## Проверка моделей на тестовой выборке

In [14]:
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

### DecisionTree

In [15]:
decision_model = DecisionTreeClassifier(random_state=1000, max_depth=4)
decision_model.fit(features_train, target_train)
predictions_test = decision_model.predict(features_test)
print('Accuracy в тестовой выборке =', accuracy_score(target_test, predictions_test))

Accuracy в тестовой выборке = 0.7993779160186625


### RandomForest

In [19]:
forest_model = RandomForestClassifier(random_state=1000, n_estimators=100)
forest_model.fit(features_train, target_train)
predictions_test = forest_model.predict(features_test)
print('Accuracy в тестовой выборке =', accuracy_score(target_test, predictions_test))

Accuracy в тестовой выборке = 0.8118195956454122


**Вывод:**
* Точность моделей DecisionTreeClassifier и RandomForestClassifier практически одинакова
* Значение точности моделей равна 0.79, то есть модель ошибается примерно в двух случаях из десяти

## Проверка модели на адекватность

Для поверки восспользуемся моделью DummyClassifier, которую используют для сравнения с более сложными.

In [17]:
dummy_model = DummyClassifier(random_state=1000)
dummy_model.fit(features_train, target_train)
predictions_test = dummy_model.predict(features_test)
print('Accuracy в тестовой выборке =', accuracy_score(target_test, predictions_test))

Accuracy в тестовой выборке = 0.6796267496111975


**Вывод:**
При сравнении моделей DecisionTreeClassifier(точность 0,79) и RandomForestClassifier(точность 0,79) с простой моделью DummyClassifier проверили адекватность результатов, и убедились, что наши модели точнее.